# GPU programming: Parallel Reduction

Implementation of different optimisation methods for parallel reduction of a large array with CUDA, as presented in [this presentation](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf) of Mark Harris.

Implemented Kernels:
* Interleaved addressing with high **divergence**
* Interleaved addressing with **bank conflicts**
* Sequential addressing without **bank conflicts**

Also a version with **Thrust** library.

Execute the cells below to run the code in google colab environment.

In [50]:
! pip install git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-9znw1z6_
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-9znw1z6_
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=69694894dddb35da0df212cf4c368ed118ba5796147ea193ffde451ff3087e5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrh9r5rt/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [51]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [53]:
%%cu

#include <stdlib.h>
#include <stdio.h>
#include <cuda.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <math.h>
#include <time.h>

void print_array(char *chaine, float *tab, int len){
   int k;
   printf("\nIn array: %i elements", len);
   printf("\nLes 10 premiers de %s: \n",chaine);
   for (k=0; k<10; k++) 
      printf("%.2f ",tab[k]);
   printf("\nLes 10 derniers: \n");
   for (k=len-10; k<len; k++) 
      printf("%.2f ",tab[k]);
   printf("\n");
}

////////////////////////////////////////////////////////////////////////////////

enum reduce_type { DIVERGENT_BRANCH, BANK_CONFLICT, NO_BANK_CONFLICT };

/* -------- KERNEL WITH DIVERGENT BRANCHING -------- */
__global__ void reduce_kernel(float * d_out, float * d_in)
{
  extern __shared__ float sdata[];
 
  unsigned int tid = threadIdx.x;
  unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
  sdata[tid] = d_in[i];
  __syncthreads();
 
  for(unsigned int s = 1; s < blockDim.x; s *= 2){
    if(tid % (2*s) == 0){
        sdata[tid] += sdata[tid+s];
    }
    __syncthreads();
  }

  // only thread 0 writes result, as thread
  if (tid == 0){
    //printf("\nblockIdx %i: %f", blockIdx.x, sdata[tid]);
    d_out[blockIdx.x] = sdata[0];
  }
}

/* -------- KERNEL WITH BANK CONFLICTS -------------- */
__global__ void reduce_kernel_bank_conflicts(float * d_out, float * d_in)
{
  extern __shared__ float sdata[];
 
  unsigned int tid = threadIdx.x;
  unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
  sdata[tid] = d_in[i];
  __syncthreads();
 
  for (unsigned int s=1; s < blockDim.x; s *= 2) {
    int index = 2 * s * tid;
    if (index < blockDim.x) {
      sdata[index] += sdata[index + s];
    }
    __syncthreads();
  }

  // only thread 0 writes result, as thread
  if (tid == 0){
    //printf("\nblockIdx %i: %f", blockIdx.x, sdata[tid]);
    d_out[blockIdx.x] = sdata[0];
  }
}

/* -------- KERNEL WITHOUT BANK CONFLICTS -------------- */
__global__ void reduce_kernel_without_bank_conflicts(float * d_out, float * d_in)
{
  extern __shared__ float sdata[];
 
  unsigned int tid = threadIdx.x;
  unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
  sdata[tid] = d_in[i];
  __syncthreads();
 
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }

  // only thread 0 writes result, as thread
  if (tid == 0){
    //printf("\nblockIdx %i: %f", blockIdx.x, sdata[tid]);
    d_out[blockIdx.x] = sdata[0];
  }
}

void apply_kernel(int num_blocks, int num_threads, float * d_out, float * d_in, reduce_type reduce_flag)
{
  switch (reduce_flag)
  {
      case DIVERGENT_BRANCH:
        reduce_kernel<<<num_blocks, num_threads, sizeof(float)*num_threads>>>(d_out, d_in);
        break;
      case BANK_CONFLICT:
        reduce_kernel_bank_conflicts<<<num_blocks, num_threads, sizeof(float)*num_threads>>>(d_out, d_in);
        break;
      case NO_BANK_CONFLICT:
        reduce_kernel_without_bank_conflicts<<<num_blocks, num_threads, sizeof(float)*num_threads>>>(d_out, d_in);
        break;
      default:
        reduce_kernel<<<num_blocks, num_threads, sizeof(float)*num_threads>>>(d_out, d_in);
  }
}

////////////////////////////////////////////////////////////////////////////////

/* -------- RECURSIVE KERNEL REDUCTION -------------- */
void recursive_reduction(float* d_out, float* d_in, const int size_in, const int num_threads, reduce_type reduce_flag)
{
  switch (reduce_flag)
  {
      case DIVERGENT_BRANCH:
        printf("\n---- Reduction with divergent branching ----\n");
        break;
      case BANK_CONFLICT:
        printf("\n---- Reduction with bank conflicts ----\n");
        break;
      case NO_BANK_CONFLICT:
        printf("\n---- Reduction without bank conflicts ----\n");
        break;
      default:
        printf("\n---- Reduction with divergent branching ----\n");
  }
 
  // Setting up initial number of blocks
  int num_blocks = ((size_in-1) / num_threads) + 1;
  int prev_num_blocks = size_in;

  // To store intermediate input and output array
  // we define two stuctures.
  // We will need to update them between kernel operations
  float * d_in_intermediate;
  cudaMalloc(&d_in_intermediate, sizeof(float)*size_in);
  cudaMemcpy(d_in_intermediate, d_in, sizeof(float)*size_in, cudaMemcpyDeviceToDevice);
 
  // Setting up intermediate ouput structure
  // to store intermediate results between recursive kernel calls.
  // Each block compute one element of intermediate result
  int extended_size_out = ( ((num_blocks-1)/num_threads) + 1 ) * num_threads;
  float * d_out_intermediate;
  cudaMalloc(&d_out_intermediate, sizeof(float)*extended_size_out);
  cudaMemset(d_out_intermediate, 0, sizeof(float)*extended_size_out);
 
  printf("\nInitial number of blocks: %i", num_blocks);
  // recursively solving, will run approximately log base num_threads times.
  do
  {
    apply_kernel(num_blocks, num_threads, d_out_intermediate, d_in_intermediate, reduce_flag);
    cudaDeviceSynchronize();

    // Updating input to intermediate results for recursion
    // Intermediate output become new intermediate input
    cudaFree(d_in_intermediate);
    cudaMalloc(&d_in_intermediate, sizeof(float)*extended_size_out);
    cudaMemcpy(d_in_intermediate, d_out_intermediate, sizeof(float)*extended_size_out, cudaMemcpyDeviceToDevice);

    prev_num_blocks = num_blocks;
   
    // New number of blocks required for next recursion
    num_blocks = (num_blocks-1) / num_threads + 1;
    if(num_blocks > 1 || prev_num_blocks > 1) printf("\n\nNext reduce number of blocks: %i", num_blocks);
   
    // New extended size for intermediate output
    extended_size_out = ( ((num_blocks-1)/num_threads) + 1 ) * num_threads;
   
    // Updating intermediate structure
    cudaFree(d_out_intermediate);
    cudaMalloc(&d_out_intermediate, sizeof(float)*extended_size_out);
    cudaMemset(d_out_intermediate, 0, sizeof(float)*extended_size_out);
  }
  while(num_blocks > 1);
  
  // computing rest
  if(prev_num_blocks > 1)
  apply_kernel(1, num_threads, d_out, d_in_intermediate, reduce_flag);
  else
  cudaMemcpy(d_out, d_in_intermediate, sizeof(float), cudaMemcpyDeviceToDevice);
  
  cudaFree(d_in_intermediate);
  cudaFree(d_out_intermediate);
}

////////////////////////////////////////////////////////////////////////////////

/* -------- Reduction with THRUST -------- */
void thrust_reduce(float* h_in, const int size_in)
{
  printf("\n---- Reduction with Thrust ----\n");
 
  thrust::device_vector<float> D (h_in, h_in+size_in);
  float sum_t;
 
  clock_t start = clock();
  sum_t = thrust::reduce(D.begin(), D.end());
  clock_t end = clock();
  printf("\nReduction time: %.f µs\n", (double) (end-start));
  printf("\nSum: %.f\n", sum_t);
}

////////////////////////////////////////////////////////////////////////////////

/* -------- MAIN -------- */
int main(void)
{
  // Setting num_threads
  const int num_threads = 128;
 
  // Setting input array
  const int size_in = pow(2, 22) + 1;
 
  float * h_in = (float *)malloc(size_in*sizeof(float));
  for (int i = 0; i < size_in; i++) h_in[i] = 1.0f;
 
  print_array("h_in", h_in, size_in);

  // Setting up an extended size if input array is not a multiple of num_threads
  // We will extend the memory on the gpu initialized with value of 0 
  int extended_size_in = ( ((size_in-1)/num_threads) + 1 ) * num_threads;
 
  // Memory allocation on device for input and ouput
  float * d_in;
  cudaMalloc(&d_in, sizeof(float)*extended_size_in);
  cudaMemset(d_in, 0, sizeof(float)*extended_size_in);
 
  float * d_out;
  cudaMalloc((void**)&d_out, sizeof(float));

  // Copy input on device 
  cudaMemcpy(d_in, h_in, sizeof(float)*size_in, cudaMemcpyHostToDevice);
 
  // Reduction with recursive kernel calls
  clock_t start = clock();
  recursive_reduction(d_out, d_in, extended_size_in, num_threads, DIVERGENT_BRANCH);
  clock_t end = clock();
  printf("\n\nReduction time: %.f µs\n", (double) (end-start));
 
  start = clock();
  recursive_reduction(d_out, d_in, extended_size_in, num_threads, BANK_CONFLICT);
  end = clock();
  printf("\n\nReduction time: %.f µs\n", (double) (end-start));
 
  start = clock();
  recursive_reduction(d_out, d_in, extended_size_in, num_threads, NO_BANK_CONFLICT);
  end = clock();
  printf("\n\nReduction time: %.f µs\n", (double) (end-start));
 
  float result;
  cudaMemcpy(&result, d_out, sizeof(float), cudaMemcpyDeviceToHost);
  printf("\n\nTotal sum of elements in array is: %.f\n", result);
 
  // Version with thrust
  thrust_reduce(h_in, size_in);
 
  cudaFree(d_in);
  cudaFree(d_out);

  free(h_in);
}


In array: 4194305 elements
Les 10 premiers de h_in: 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
Les 10 derniers: 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

---- Reduction with divergent branching ----

Initial number of blocks: 32769

Next reduce number of blocks: 257

Next reduce number of blocks: 3

Next reduce number of blocks: 1

Reduction time: 1509 µs

---- Reduction with bank conflicts ----

Initial number of blocks: 32769

Next reduce number of blocks: 257

Next reduce number of blocks: 3

Next reduce number of blocks: 1

Reduction time: 1245 µs

---- Reduction without bank conflicts ----

Initial number of blocks: 32769

Next reduce number of blocks: 257

Next reduce number of blocks: 3

Next reduce number of blocks: 1

Reduction time: 1189 µs


Total sum of elements in array is: 4194305

---- Reduction with Thrust ----

Reduction time: 168 µs

Sum: 4194305

